In [54]:
import pandas as pd
import numpy as np

In [55]:
df = pd.read_csv("fund_sheet.csv")

In [56]:
fund_categories = df["Sub Category"].unique()
len(fund_categories) 

51

In [57]:
df.head(5)

,Name,Sub Category,NSE Benchmark,BSE Benchmark,Crisil Benchmark,MSCI Benchmark,Plan,AUM,expense_ratio,lockin,...,volatility,category_st_dev,maximum_drawdown,percentage_equity_holding,percentage_debt_holding,percentage_largecap_holding,percentage_midcap_holding,percentage_smallcap_holding,percentage_concetration__top_5_holdings,percentage_cash_holding
0,ICICI Pru Equity & Debt Fund,Aggressive Hybrid Fund,NIFTY 50 Hybrid Composite Debt 65:35 Index,NaN,CRISIL Hybrid 35+65 Aggressive Index,NaN,Growth,40203.38,0.98,0,...,10.35,11.25,30.70,63.11,23.03,54.79,3.29,5.04,28.90,5.83
1,Quant Absolute Fund,Aggressive Hybrid Fund,NIFTY 50 Hybrid Composite Debt 65:35 Index,NaN,CRISIL Hybrid 35+65 Aggressive Index,NaN,Growth,2250.71,0.70,0,...,13.85,11.25,28.69,77.11,21.16,75.45,0.68,0.99,37.63,1.47
2,Kotak Equity Hybrid Fund,Aggressive Hybrid Fund,NIFTY 50 Hybrid Composite Debt 65:35 Index,NaN,CRISIL Hybrid 35+65 Aggressive Index,NaN,Growth,6714.55,0.45,0,...,11.23,11.25,32.14,73.19,25.45,42.67,19.38,11.13,21.94,1.27
3,DSP Equity & Bond Fund,Aggressive Hybrid Fund,NIFTY 50 Hybrid Composite Debt 65:35 Index,NaN,CRISIL Hybrid 35+65 Aggressive Index,NaN,Growth,10327.37,0.73,0,...,9.98,11.25,28.81,68.53,28.27,43.31,15.04,10.17,23.86,3.20
4,Canara Rob Equity Hybrid Fund,Aggressive Hybrid Fund,NIFTY 50 Hybrid Composite Debt 65:35 Index,NaN,CRISIL Hybrid 35+65 Aggressive Index,NaN,Growth,10873.25,0.58,0,...,10.89,11.25,25.69,70.87,24.40,48.91,12.87,9.10,19.27,4.73


Filtering out only the funds with Nifty 500 as benchmark

In [58]:
df_nifty_500 = df[df['NSE Benchmark'] == 'NIFTY 500']
df_sorted = df_nifty_500.sort_values('absolute_returns__3m', ascending=False)

nifty_500_returns = {
    'absolute_returns__3m': -1.21,
    'absolute_returns__6m': 10.63,
    'absolute_returns__1y': 31.23,
    'cagr_3y': 13.42,
    'cagr_5y': 18.52,
    'cagr_10y': 12.90
}

portfolio_sizes = [5, 10, 25, 50]

def calculate_mean(df, column_name):
    valid_data = df[column_name][df[column_name] != 0]
    if len(valid_data) >= 0.75 * len(df):
        return valid_data.mean()
    return f"Only {len(valid_data)}/{len(df)}"

def calculate_aum_average(df):
    valid_data = df['AUM'][df['AUM'] != 0]
    return valid_data.mean()

def calculate_vol_average(df):
    valid_data = df['volatility'][df['volatility'] != 0]
    return valid_data.mean()
    
table_1 = []

for size in portfolio_sizes:
    top_funds = df_sorted.head(size)
    aum_average = calculate_aum_average(top_funds)
    volatility = calculate_vol_average(top_funds)

    row_1 = [
        size,
        aum_average,
        volatility, 
        calculate_mean(top_funds, 'absolute_returns__3m'),
        calculate_mean(top_funds, 'absolute_returns__6m'),
        calculate_mean(top_funds, 'absolute_returns__1y'),
        calculate_mean(top_funds, 'cagr_3y'),
        calculate_mean(top_funds, 'cagr_5y'),
        calculate_mean(top_funds, 'cagr_10y')
    ]
    table_1.append(row_1)

columns = ['Portfolio size (funds)', 'AUM average', 'Volatility', '3m absolute', '6m absolute', '1y absolute', '3y cagr', '5y cagr', '10y cagr']

df_table_1 = pd.DataFrame(table_1, columns=columns)

In [59]:
df_table_1.to_csv("sort_by_3m_t1.csv")

In [60]:
nifty_500_returns = {
    'absolute_returns__3m': -1.21,
    'absolute_returns__6m': 10.63,
    'absolute_returns__1y': 31.23,
    'cagr_3y': 13.42,
    'cagr_5y': 18.52,
    'cagr_10y': 12.90
}

df_table_2 = df_table_1.copy()

column_mapping = {
    '3m absolute': 'absolute_returns__3m',
    '6m absolute': 'absolute_returns__6m',
    '1y absolute': 'absolute_returns__1y',
    '3y cagr': 'cagr_3y',
    '5y cagr': 'cagr_5y',
    '10y cagr': 'cagr_10y'
}

def calculate_excess_return(value, column):
    if isinstance(value, str) and "Only" in value:
        return '-'
    else:
        nifty_return = nifty_500_returns[column_mapping[column]]
        return value - nifty_return

for column in df_table_1.columns:
    if column in column_mapping:
        df_table_2[column] = df_table_1[column].apply(lambda x: calculate_excess_return(x, column))

In [61]:
df_table_2.to_csv("sort_by_3m_t2.csv")